In [1]:
import pandas as pd
import numpy as np
import torch
import utils.print as print_f
import os

from utils.engine import evaluate, get_iou_types
from utils.plot import plot_losses, plot_ap_ars

from models.utils import get_model_size_in_MB
from models.build import create_model_from_setup
from models.setup import ModelSetup
from models.train import TrainingInfo
from utils.save import check_best, end_train, get_data_from_metric_logger
from data.load import get_datasets, get_dataloaders
from IPython.display import clear_output
from utils.eval import get_ap_ar, get_ap_ar_for_train_val
from utils.train import get_optimiser, get_lr_scheduler, print_params_setup, get_coco_eval_params, get_dynamic_loss
from utils.init import reproducibility, clean_memory_get_device
from data.paths import MIMIC_EYE_PATH
# from datetime import datetime
# from models.dynamic_loss import DynamicWeightedLoss
from utils.engine import train_one_epoch, evaluate
from torch import optim
from data.paths import MIMIC_EYE_PATH
from models.load import get_trained_model
# from data.load import seed_worker, get_dataloader_g


## Suppress the assignement warning from pandas.r
pd.options.mode.chained_assignment = None  # default='warn'

## Supress user warning
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [21]:
from enum import Enum

class TrainedModels(Enum):
    continue_training = "val_lesion-detection_ap_0_1226_test_lesion-detection_ap_0_1372_epoch30_03-09-2023 11-58-37_chexpert_without_fix"
    with_fix = "val_lesion-detection_ap_0_0005_test_lesion-detection_ap_0_0010_epoch1_03-09-2023 02-08-50_chexpert_with_fix"


In [22]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [23]:
get_epoch_from_model = lambda select_model: int(([substr for substr in select_model.value.split("_") if "epoch" in substr][0]).replace("epoch", ""))
get_model_name = lambda select_model: str(select_model).split(".")[-1]

In [24]:
select_model = TrainedModels.with_fix

In [25]:
model, train_info, optimizer, dynamic_loss_weight = get_trained_model(
        select_model,
        device,
    )

model.to(device)
model.train()

print()

Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3
Using SGD as optimizer with lr=0.001



In [18]:
dynamic_loss_weight.params.keys()

dict_keys(['lesion-detection_performer-object_detection_loss_box_reg', 'lesion-detection_performer-object_detection_loss_classifier', 'lesion-detection_performer-object_detection_loss_mask', 'lesion-detection_performer-object_detection_loss_objectness', 'lesion-detection_performer-object_detection_loss_rpn_box_reg'])

In [20]:
for k in dynamic_loss_weight.params.keys():
    print(f"{k}: {dynamic_loss_weight.params[k].item()}")

lesion-detection_performer-object_detection_loss_box_reg: 0.4498993158340454
lesion-detection_performer-object_detection_loss_classifier: 0.4715196192264557
lesion-detection_performer-object_detection_loss_mask: 1.0
lesion-detection_performer-object_detection_loss_objectness: 0.3527882397174835
lesion-detection_performer-object_detection_loss_rpn_box_reg: 0.1935456395149231


In [26]:
for k in dynamic_loss_weight.params.keys():
    print(f"{k}: {dynamic_loss_weight.params[k].item()}")

lesion-detection_performer-object_detection_loss_box_reg: 0.45166999101638794
lesion-detection_performer-object_detection_loss_classifier: 0.7384166717529297
lesion-detection_performer-object_detection_loss_mask: 1.0
lesion-detection_performer-object_detection_loss_objectness: 0.8888599276542664
lesion-detection_performer-object_detection_loss_rpn_box_reg: 0.45957323908805847


In [ ]:
A